In [1]:
%reload_ext dotenv
%dotenv .env

In [2]:
import os
import re

from mechanicalsoup import StatefulBrowser

browser = StatefulBrowser()

#### Get all tram line timetable URLs

In [3]:
browser.open(os.environ.get('TIMETABLE_URL'))

<Response [200]>

In [4]:
tram_lines = {
    item.text.strip(): item.get("href")[1:]
    for item in browser.page.findAll("a", class_=lambda x: x and x.startswith("linia"))
    if len(item.text.strip()) <= 2
}

tram_lines

{'1': '?lang=PL&linia=1',
 '3': '?lang=PL&linia=3',
 '4': '?lang=PL&linia=4',
 '5': '?lang=PL&linia=5',
 '8': '?lang=PL&linia=8',
 '9': '?lang=PL&linia=9',
 '10': '?lang=PL&linia=10',
 '11': '?lang=PL&linia=11',
 '13': '?lang=PL&linia=13',
 '14': '?lang=PL&linia=14',
 '16': '?lang=PL&linia=16',
 '17': '?lang=PL&linia=17',
 '18': '?lang=PL&linia=18',
 '20': '?lang=PL&linia=20',
 '21': '?lang=PL&linia=21',
 '22': '?lang=PL&linia=22',
 '24': '?lang=PL&linia=24',
 '50': '?lang=PL&linia=50',
 '52': '?lang=PL&linia=52',
 '62': '?lang=PL&linia=62',
 '64': '?lang=PL&linia=64',
 '69': '?lang=PL&linia=69',
 '72': '?lang=PL&linia=72'}

## Line 1

#### Line variants

In [5]:
browser.open(os.environ.get('TIMETABLE_URL') + tram_lines['1'])

<Response [200]>

In [6]:
variant_urls = {
    item.text.strip(): item.get("href")[1:]
    for item in browser.page.findAll("table")[7].findAll("a")
}

variant_urls

{'Rondo Hipokratesa - Cichy Kącik': '?lang=PL&akcja=index&rozklad=20240504&linia=1__1',
 'Cichy Kącik - Rondo Hipokratesa': '?lang=PL&akcja=index&rozklad=20240504&linia=1__2'}

#### First variant's stops

In [7]:
browser.open(os.environ.get('TIMETABLE_URL') + list(variant_urls.values())[0])

<Response [200]>

In [8]:
stop_urls = {
    item.text.strip(): item.find("a") and item.find("a").get("href")[1:]
    for item in browser.page.findAll("table")[10].findAll("td")
    if item.text.strip()
}

stop_urls

{'Rondo Hipokratesa 02': '?lang=PL&rozklad=20240504&linia=1__1__1',
 'DH Wanda 02': '?lang=PL&rozklad=20240504&linia=1__1__2',
 'Rondo Kocmyrzowskie im. Ks. Gorzelanego 02': '?lang=PL&rozklad=20240504&linia=1__1__3',
 'Bieńczycka 01': '?lang=PL&rozklad=20240504&linia=1__1__4',
 'Rondo Czyżyńskie 04': '?lang=PL&rozklad=20240504&linia=1__1__5',
 'Centralna 02': '?lang=PL&rozklad=20240504&linia=1__1__6',
 'Rondo 308. Dywizjonu 02': '?lang=PL&rozklad=20240504&linia=1__1__7',
 'Ogród Doświadczeń 02': '?lang=PL&rozklad=20240504&linia=1__1__8',
 'TAURON Arena Kraków al. Pokoju 02': '?lang=PL&rozklad=20240504&linia=1__1__9',
 'Dąbie 02': '?lang=PL&rozklad=20240504&linia=1__1__10',
 'Ofiar Dąbia 02': '?lang=PL&rozklad=20240504&linia=1__1__11',
 'Fabryczna 02': '?lang=PL&rozklad=20240504&linia=1__1__12',
 'Francesco Nullo 02': '?lang=PL&rozklad=20240504&linia=1__1__13',
 'Teatr Variété 02': '?lang=PL&rozklad=20240504&linia=1__1__14',
 'Rondo Grzegórzeckie 02': '?lang=PL&rozklad=20240504&linia=1_

#### First stop's departures

In [9]:
browser.open(os.environ.get('TIMETABLE_URL') + list(stop_urls.values())[0])

<Response [200]>

In [10]:
for item in browser.page.findAll("table")[13].findAll("tr")[1:-2]:
    cells = item.findAll("td")

    hour = int(cells[0].text)
    weekday_minutes = [int(minute) for minute in cells[1].text.strip().split()]
    saturday_minutes = [int(minute) for minute in cells[2].text.strip().split()]
    sunday_minutes = [int(minute) for minute in cells[3].text.strip().split()]

    print(hour, weekday_minutes, saturday_minutes, sunday_minutes, sep="\t")

3	[]	[]	[]
4	[48]	[54]	[]
5	[1, 15, 32, 47]	[14, 34, 54]	[1, 31]
6	[3, 17, 33, 47]	[14, 34, 54]	[1, 31]
7	[3, 18, 33, 48]	[14, 34, 54]	[1, 31]
8	[3, 18, 33, 48]	[13, 33, 53]	[3, 33]
9	[3, 18, 33, 48]	[13, 33, 53]	[3, 33, 53]
10	[3, 18, 33, 48]	[13, 33, 53]	[13, 33, 53]
11	[3, 18, 33, 48]	[13, 33, 53]	[13, 33, 53]
12	[3, 18, 33, 48]	[13, 33, 53]	[13, 33, 53]
13	[3, 16, 32, 47]	[13, 33, 53]	[13, 33, 53]
14	[4, 21, 36, 51]	[13, 33, 53]	[13, 33, 53]
15	[6, 21, 36, 51]	[13, 33, 53]	[13, 33, 53]
16	[6, 21, 36, 51]	[13, 33, 53]	[13, 33, 53]
17	[6, 21, 36, 51]	[13, 33, 53]	[13, 33, 53]
18	[6, 21, 36, 53]	[13, 33, 53]	[13, 33, 53]
19	[10, 25, 40, 57]	[13, 33, 53]	[13, 33, 53]
20	[10, 23, 38, 53]	[14, 34, 54]	[14, 34, 54]
21	[12, 32, 51]	[14, 34, 54]	[14, 34, 54]
22	[13, 33, 54]	[14, 34]	[14, 34]
23	[]	[4]	[4]
0	[]	[]	[]


## Line 50

#### Line variants

In [11]:
browser.open(os.environ.get('TIMETABLE_URL') + tram_lines['50'])

<Response [200]>

In [12]:
variant_urls = {
    item.text.strip(): item.get("href")[1:]
    for item in browser.page.findAll("table")[7].findAll("a")
}

variant_urls

{'Górka Narodowa P+R - Borek Fałęcki': '?lang=PL&akcja=index&rozklad=20240504&linia=50__1',
 'Borek Fałęcki - Górka Narodowa P+R': '?lang=PL&akcja=index&rozklad=20240504&linia=50__2'}

#### First variant's stops

In [13]:
browser.open(os.environ.get('TIMETABLE_URL') + list(variant_urls.values())[0])

<Response [200]>

In [14]:
stop_urls = {
    item.text.strip(): item.find("a") and item.find("a").get("href")[1:]
    for item in browser.page.findAll("table")[10].findAll("td")
    if item.text.strip()
}

stop_urls

{'Górka Narodowa P+R 01': '?lang=PL&rozklad=20240504&linia=50__1__1',
 'Papierni Prądnickich 02': '?lang=PL&rozklad=20240504&linia=50__1__2',
 'Kuźnicy Kołłątajowskiej 02': '?lang=PL&rozklad=20240504&linia=50__1__3',
 'Bociana 02': '?lang=PL&rozklad=20240504&linia=50__1__4',
 'Siewna Wiadukt 02': '?lang=PL&rozklad=20240504&linia=50__1__5',
 'Górnickiego 01': '?lang=PL&rozklad=20240504&linia=50__1__6',
 'Białoprądnicka 01': '?lang=PL&rozklad=20240504&linia=50__1__7',
 'Pachońskiego P+R 02': '?lang=PL&rozklad=20240504&linia=50__1__8',
 'Krowodrza Górka P+R 02': '?lang=PL&rozklad=20240504&linia=50__1__9',
 'Bratysławska 02': '?lang=PL&rozklad=20240504&linia=50__1__10',
 'Szpital Narutowicza 02': '?lang=PL&rozklad=20240504&linia=50__1__11',
 'Dworzec Towarowy 03': '?lang=PL&rozklad=20240504&linia=50__1__12',
 'Nowy Kleparz 02': '?lang=PL&rozklad=20240504&linia=50__1__13',
 'Pędzichów 02': '?lang=PL&rozklad=20240504&linia=50__1__14',
 'Stary Kleparz 03': '?lang=PL&rozklad=20240504&linia=50_

#### First stop's departures

In [15]:
browser.open(os.environ.get('TIMETABLE_URL') + list(stop_urls.values())[0])

<Response [200]>

In [16]:
for item in browser.page.findAll("table")[13].findAll("tr")[1:-2]:
    cells = item.findAll("td")

    hour = int(cells[0].text)
    print()
    weekday_minutes = [int(re.sub("[^0-9]", "", minute)) for minute in cells[1].text.strip().split()]
    saturday_minutes = [int(re.sub("[^0-9]", "", minute)) for minute in cells[2].text.strip().split()]
    sunday_minutes = [int(re.sub("[^0-9]", "", minute)) for minute in cells[3].text.strip().split()]

    print(hour, weekday_minutes, saturday_minutes, sunday_minutes, sep="\t")


3	[]	[]	[]

4	[37, 54]	[40]	[47]

5	[12, 23, 38, 45, 53]	[0, 20, 40]	[17, 47]

6	[0, 7, 15, 22, 30, 38, 45, 53]	[0, 20, 40]	[17, 47]

7	[0, 8, 15, 23, 30, 38, 45, 53]	[0, 20, 40, 48, 59]	[17, 38, 59]

8	[0, 8, 15, 24, 32, 39, 47, 54]	[8, 19, 28, 39, 48, 59]	[19, 39, 59]

9	[2, 9, 17, 24, 32, 39, 47, 54]	[8, 19, 28, 39, 48, 59]	[19, 39, 59]

10	[2, 9, 17, 24, 32, 39, 47, 54]	[8, 19, 28, 39, 48, 59]	[19, 39, 59]

11	[2, 9, 17, 24, 32, 39, 47, 54]	[8, 19, 28, 39, 48, 59]	[19, 39, 59]

12	[2, 9, 17, 24, 32, 39, 46, 54]	[8, 19, 28, 39, 48, 59]	[19, 39, 59]

13	[2, 9, 18, 25, 33, 39, 48, 56]	[8, 19, 28, 39, 48, 59]	[19, 39, 48, 59]

14	[3, 11, 18, 26, 33, 41, 48, 56]	[8, 19, 28, 39, 48, 59]	[8, 19, 28, 39, 48, 59]

15	[3, 11, 18, 26, 33, 41, 48, 56]	[8, 19, 28, 39, 48, 59]	[8, 19, 28, 39, 48, 59]

16	[3, 11, 18, 26, 33, 39, 48, 56]	[8, 19, 28, 39, 48, 59]	[8, 19, 28, 39, 48, 59]

17	[3, 11, 18, 26, 33, 41, 48, 55]	[8, 19, 28, 39, 48, 59]	[8, 19, 28, 39, 48, 59]

18	[3, 10, 17, 25, 33, 40, 4